In [ ]:

!pip install py-solc-x web3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.4/331.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.8/170.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.4 MB/s eta 0:00:00


In [ ]:
!pip install eth-tester
!pip install web3 py-solc-x



In [ ]:

from solcx import compile_source, install_solc
from web3 import Web3

# Install specific Solidity compiler version
install_solc("0.8.0")


<Version('0.8.0')>

In [ ]:

lottery_contract_source = '''
// SPDX-License-Identifier: GPL-3.0
pragma solidity >=0.7.0 <0.9.0;

contract Lottery {
    address public manager;
    address payable[] public players;
    address payable public winner;

    constructor() {
        manager = msg.sender;
    }

    function participate() public payable {
        require(msg.value == 1 ether, "Please pay 1 ether only!");
        players.push(payable(msg.sender));
    }

    function getBalance() public view returns (uint) {
        require(manager == msg.sender, "You are not the manager!");
        return address(this).balance;
    }

    uint private manualRandomNumber;

    function setManualRandomNumber(uint _manualRandomNumber) external {
        require(msg.sender == manager, "You are not the manager!");
        manualRandomNumber = _manualRandomNumber;
    }

    function getRandomNumber() internal view returns (uint) {
        if (manualRandomNumber != 0) {
            return manualRandomNumber;
        }
        return uint(keccak256(abi.encodePacked(blockhash(block.number - 1), block.timestamp)));
    }

    function pickWinner() public {
        require(msg.sender == manager, "You are not the manager!");
        require(players.length >= 3, "There are less than 3 players!");
        uint r = getRandomNumber();
        uint index = r % players.length;
        winner = players[index];
        winner.transfer(address(this).balance);

        // ✅ Reset players array
        delete players;
    }
}
'''



In [ ]:

compiled_sol = compile_source(lottery_contract_source, output_values=["abi", "bin"])
contract_id, contract_interface = compiled_sol.popitem()

abi = contract_interface["abi"]
bytecode = contract_interface["bin"]

print(" Contract Compiled Successfully")


✅ Contract Compiled Successfully


In [ ]:

w3 = Web3(Web3.EthereumTesterProvider())
w3.is_connected()


Package 'py-evm' not found.
Package 'py-evm' not found.
Package 'py-evm' not found.
Package 'py-evm' not found.


/usr/local/lib/python3.12/dist-packages/eth_tester/backends/__init__.py:45: UserWarning: Ethereum Tester: No backend was explicitly set, and no *full* backends were available.  Falling back to the `MockBackend` which does not support all EVM functionality.  Please refer to the `eth-tester` documentation for information on what backends are available and how to set them.  Your py-evm package may need to be updated.
  backend_class = get_chain_backend_class()
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


True

In [ ]:

# Adding players
for i in range(1, 4):
    lottery_instance.functions.participate().transact({"from": w3.eth.accounts[i], "value": w3.to_wei(1, "ether")})

# Manager picks winner
lottery_instance.functions.setManualRandomNumber(42).transact({"from": acct})
lottery_instance.functions.pickWinner().transact({"from": acct})

print("🏆 Winner Address:", lottery_instance.functions.winner().call())


In [ ]:

acct = w3.eth.accounts[0]
Lottery = w3.eth.contract(abi=abi, bytecode=bytecode)

tx_hash = Lottery.constructor().transact({"from": acct})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

lottery_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

print("✅ Contract Deployed at:", tx_receipt.contractAddress)
